In [38]:
import asyncio # 비동기처리를 위함
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

In [78]:
def expand_shadow_element(element):
    shadow_root = driver.execute_script('return arguments[0].shadowRoot', element)
    return shadow_root

In [83]:
# arr에는 shadow DOM 각각의 상위 태그 및 selector 정보를 순서대로 넣기
# target에는 최종적으로 찾고자하는 selector 넣기
def find_element_in_shadowDOMs(arr, target):
    curr_root = driver
    root, shadow_root = None, None
    for info in arr:
        root = curr_root.find_element_by_css_selector(info)
        shadow_root = expand_shadow_element(root)
        curr_root = shadow_root
        print(curr_root)
    
    return curr_root.find_element_by_css_selector(target)

In [86]:
# chrome://settings/content/javascript 페이지에서 사용할것.
def toggleJavascriptAllow():
    arr = ["settings-ui",
       "#container > settings-main", 
       "settings-basic-page", 
       "#basicPage > settings-section.expanded > settings-privacy-page", 
       "settings-subpage > category-default-setting", 
       "settings-toggle-button"]
    find_element_in_shadowDOMs(arr, "#outerRow > #control").click()

In [122]:
# naver blog에서 1~10 페이지까지의 블로그 url 크롤링
def getBlogsURL(driver, term):
    pageIndexes = range(1, 92, 10) # 1~10페이지
    blogs = []
    for pageIndex in pageIndexes:
        driver.get(F"https://search.naver.com/search.naver?where=post&query={term}&start={pageIndex}")
        dirtyURLs = driver.find_elements_by_class_name("sh_blog_top > dl > .txt_block > .inline > .url")
        for url in dirtyURLs:
            blogs.append( url.get_attribute("href") )
    return blogs
        

In [ ]:
chrome_options = Options()
chrome_options.add_argument("--window-size=1920,1080")

driver = webdriver.Chrome("./chromedriver", chrome_options=chrome_options)

In [124]:
blogsURLs = list(set(getBlogsURL(driver, "test")))

In [125]:
blogsURLs[:10]

['https://www.redstate.com/nick-arama/2020/09/28/trump-calls-for-a-drug-test-prior-to-the-first-debate-biden-teams-response-is-classic-deflection/',
 'https://blog.naver.com/sciport2016?Redirect=Log&logNo=222076288151',
 'https://blog.naver.com/mt_98?Redirect=Log&logNo=222066967220',
 'https://blog.naver.com/12asap?Redirect=Log&logNo=222056269522',
 'https://blog.naver.com/desireofage?Redirect=Log&logNo=222093793433',
 'https://tiasilrans.blog.me/222038701622',
 'https://blog.naver.com/yorob?Redirect=Log&logNo=222079160159',
 'https://blog.naver.com/eunyo20?Redirect=Log&logNo=221997272748',
 'https://blog.naver.com/zzamstop?Redirect=Log&logNo=222098822468',
 'https://blog.naver.com/desireofage?Redirect=Log&logNo=222092473666']